## Assignment 3 - Multi Layer Perceptron
Implement a Multi Layer Perceptron (MLP) using raw PyTorch Tensor operations. Avoid using APIs such as torch.nn or torch.functional. You are also
not allowed to use neither torch.data nor torchvision.transforms. You can
use torchvision.datasets.MNIST.
The MLP architecture should consist of 784 input neurons, 100 hidden neurons, and 10 output neurons. 

In [ ]:
from typing import Tuple
import numpy as np
import torch
from torch import Tensor
from torchvision.datasets import MNIST
from tqdm import tqdm

Try to run your model both on CPU and GPU and check the runtime. If you do
not have a GPU available, use Google Colab and copy your script in a notebook
cell (your script should be able to run on both CPU and GPU depending on a
device parameter passed from the outside).


In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
        # For multi-gpu workstations, PyTorch will use the first available GPU (cuda:0), unless specified otherwise
        # (cuda:1).
    if torch.backends.mps.is_available():
        return torch.device('mos')
    return torch.device('cpu')

In [ ]:
def collate(x) -> Tensor:
    if isinstance(x, (tuple, list)):
        if isinstance(x[0], Tensor):
            return torch.stack(x)
        return torch.tensor(x)
    raise "Not supported yet"
    # see torch\utils\data\_utils\collate.py

In [ ]:
def to_one_hot(x: Tensor) -> Tensor:
    return torch.eye(x.max() + 1)[x]

In [ ]:
def load_mnist(path: str = "./data", train: bool = True, pin_memory: bool = True):
    mnist_raw = MNIST(path, download=True, train=train)
    mnist_data = []
    mnist_labels = []
    for image, label in mnist_raw:
        tensor = torch.from_numpy(np.array(image))
        mnist_data.append(tensor)
        mnist_labels.append(label)

    mnist_data = collate(mnist_data).float()  # shape 60000, 28, 28
    mnist_data = mnist_data.flatten(start_dim=1)  # shape 60000, 784
    mnist_data /= mnist_data.max()  # normalize the data
    mnist_labels = collate(mnist_labels)  # shape 60000
    if train:
        mnist_labels = to_one_hot(mnist_labels)  # shape 60000, 10
    if pin_memory:
        return mnist_data.pin_memory(), mnist_labels.pin_memory()
    return mnist_data, mnist_labels

In [ ]:
def forward(x: Tensor, w: Tensor, b: Tensor) -> Tensor:
    return x @ w + b

Using 2 activation functions for best results

In [ ]:
def softmax(z: Tensor) -> Tensor: # for the last layer
    return z.softmax(dim=1)

def sigmoid(z: Tensor): # for the hidden layer
    return z.sigmoid()

In [ ]:
def backward(x: Tensor, y: Tensor, y1_hat: Tensor, y2_hat: Tensor, w2: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    # Compute the error in the final layer
    error_layer2 = y2_hat - y # Shape: (batch_size, 10)

    # Calculate the gradients for weights and biases in the final layer
    delta_w2 = y1_hat.T @ error_layer2 # Shape: (100, 10)
    delta_b2 = error_layer2.mean(dim=0) # Shape: (10,)

    # Compute the error in the hidden layer
    error_layer1 = y1_hat * (1 - y1_hat) * (w2 @ error_layer2.T).T # Shape: (batch_size, 100)

    # Calculate the gradients for weights and biases in the hidden layer
    delta_w1 = x.T @ error_layer1 # Shape: (784, 100)
    delta_b1 = error_layer1.mean(dim=0) # Shape: (100,)
    
    return delta_w1, delta_b1, delta_w2, delta_b2

In [ ]:
def train_batch(x: Tensor, y: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, lr: float) -> Tuple[Tensor, Tensor, Tensor, Tensor, float]:
    # Forward Propagation
    y1_hat = sigmoid(forward(x, w1, b1)) # output of the hidden layer
    y2_hat = softmax(forward(y1_hat, w2, b2)) # output of the final layer

    # calculating the loss
    loss = torch.nn.functional.cross_entropy(y2_hat, y)

    # Backward Propagation
    delta_w1, delta_b1, delta_w2, delta_b2 = backward(x, y, y1_hat, y2_hat, w2)

    # Update weights and biases using gradient descent
    w1 -= lr * delta_w1
    b1 -= lr * delta_b1
    w2 -= lr * delta_w2
    b2 -= lr * delta_b2
    return w1, b1, w2, b2, loss

In [ ]:
def train_epoch(data: Tensor, labels: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, lr: float, batch_size: int) \
        -> Tuple[Tensor, Tensor, Tensor, Tensor, float]:
    non_blocking = w1.device.type == 'cuda'
    epoch_loss = 0
    for i in range(0, data.shape[0], batch_size):
        # Extracting a batch of data and labels
        x = data[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        y = labels[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        w1, b1, w2, b2, batch_loss = train_batch(x, y, w1, b1, w2, b2, lr)
        # Accumulate batch loss to compute epoch loss
        epoch_loss += batch_loss 

        # Compute average epoch loss
        avg_epoch_loss = epoch_loss / batch_size
    return w1, b1, w2, b2, avg_epoch_loss

In [ ]:
def evaluate(data: Tensor, labels: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, batch_size: int) -> float:
    # Labels are not one hot encoded, because we do not need them as one hot.
    total_correct_predictions = 0
    total_len = data.shape[0]
    non_blocking = w1.device.type == 'cuda'
    for i in range(0, total_len, batch_size):

        # Extract a batch of data and labels
        x = data[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        y = labels[i: i + batch_size].to(w1.device, non_blocking=non_blocking)

        # Forward pass to get predicted distribution
        predicted_distribution = softmax(forward(sigmoid(forward(x, w1, b1)), w2, b2)) 

        # check torch.max documentation
        predicted_max_value, predicted_max_value_indices = torch.max(predicted_distribution, dim=1)
        # we check if the indices of the max value per line correspond to the correct label. We get a boolean mask
        # with True where the indices are the same, false otherwise
        equality_mask = predicted_max_value_indices == y
        # We sum the boolean mask, and get the number of True values in the mask. We use .item() to get the value out of
        # the tensor
        correct_predictions = equality_mask.sum().item()
        # correct_predictions = (torch.max(predicted_distribution, dim=1)[1] == y).sum().item()
        total_correct_predictions += correct_predictions

    return total_correct_predictions / total_len

In [ ]:
def train(epochs: int = 1000, device: torch.device = get_default_device()):
    print(f"Using device {device}")
    pin_memory = device.type == 'cuda'  # Check the provided references.
    # Initialize w and b
    w1 = torch.normal(0, 1 / np.sqrt(784) , (784, 100), device=device) # Xavier/Glorot initialization
    b1 = torch.zeros((1, 100), device=device)
    w2 = torch.normal(0, 1 / np.sqrt(784), (100, 10), device=device)
    b2 = torch.zeros((1, 10), device=device)
    lr = 0.001
    batch_size = 100
    eval_batch_size = 500
    data, labels = load_mnist(train=True, pin_memory=pin_memory)
    data_test, labels_test = load_mnist(train=False, pin_memory=pin_memory)
    # Set up progress bar for epochs
    epochs = tqdm(range(epochs))
    total_loss = 0
    for epoch in epochs:
        epoch_loss = 0
        w1, b1, w2, b2, epoch_loss = train_epoch(data, labels, w1, b1, w2, b2, lr, batch_size)
        total_loss += epoch_loss
        accuracy = evaluate(data_test, labels_test, w1, b1, w2, b2, eval_batch_size)
        # Update progress bar description
        epochs.set_postfix_str(f"accuracy = {accuracy}, epoch_loss = {epoch_loss}, total_loss = {total_loss}")
        if epoch % 100 == 0:
            lr *= 0.95

In [ ]:
train(300) # for GPU
train(300, torch.device('cpu'))

# All the code in one block to test in google collab

In [ ]:

from typing import Tuple
import numpy as np
import torch
from torch import Tensor
from torchvision.datasets import MNIST
from tqdm import tqdm

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
        # For multi-gpu workstations, PyTorch will use the first available GPU (cuda:0), unless specified otherwise
        # (cuda:1).
    if torch.backends.mps.is_available():
        return torch.device('mos')
    return torch.device('cpu')

def collate(x) -> Tensor:
    if isinstance(x, (tuple, list)):
        if isinstance(x[0], Tensor):
            return torch.stack(x)
        return torch.tensor(x)
    raise "Not supported yet"
    # see torch\utils\data\_utils\collate.py

def to_one_hot(x: Tensor) -> Tensor:
    return torch.eye(x.max() + 1)[x]


def load_mnist(path: str = "./data", train: bool = True, pin_memory: bool = True):
    mnist_raw = MNIST(path, download=True, train=train)
    mnist_data = []
    mnist_labels = []
    for image, label in mnist_raw:
        tensor = torch.from_numpy(np.array(image))
        mnist_data.append(tensor)
        mnist_labels.append(label)

    mnist_data = collate(mnist_data).float()  # shape 60000, 28, 28
    mnist_data = mnist_data.flatten(start_dim=1)  # shape 60000, 784
    mnist_data /= mnist_data.max()  # normalize the data
    mnist_labels = collate(mnist_labels)  # shape 60000
    if train:
        mnist_labels = to_one_hot(mnist_labels)  # shape 60000, 10
    if pin_memory:
        return mnist_data.pin_memory(), mnist_labels.pin_memory()
    return mnist_data, mnist_labels

def forward(x: Tensor, w: Tensor, b: Tensor) -> Tensor:
    return x @ w + b

def softmax(z: Tensor) -> Tensor: # for the last layer
    return z.softmax(dim=1)

def sigmoid(z: Tensor): # for the hidden layer
    return z.sigmoid()

def backward(x: Tensor, y: Tensor, y1_hat: Tensor, y2_hat: Tensor, w2: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
    # Compute the error in the final layer
    error_layer2 = y2_hat - y # Shape: (batch_size, 10)

    # Calculate the gradients for weights and biases in the final layer
    delta_w2 = y1_hat.T @ error_layer2 # Shape: (100, 10)
    delta_b2 = error_layer2.mean(dim=0) # Shape: (10,)

    # Compute the error in the hidden layer
    error_layer1 = y1_hat * (1 - y1_hat) * (w2 @ error_layer2.T).T # Shape: (batch_size, 100)

    # Calculate the gradients for weights and biases in the hidden layer
    delta_w1 = x.T @ error_layer1 # Shape: (784, 100)
    delta_b1 = error_layer1.mean(dim=0) # Shape: (100,)
    
    return delta_w1, delta_b1, delta_w2, delta_b2


def train_batch(x: Tensor, y: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, lr: float) -> Tuple[Tensor, Tensor, Tensor, Tensor, float]:
    # Forward Propagation
    y1_hat = sigmoid(forward(x, w1, b1)) # output of the hidden layer
    y2_hat = softmax(forward(y1_hat, w2, b2)) # output of the final layer

    # calculating the loss
    loss = torch.nn.functional.cross_entropy(y2_hat, y)

    # Backward Propagation
    delta_w1, delta_b1, delta_w2, delta_b2 = backward(x, y, y1_hat, y2_hat, w2)

    # Update weights and biases using gradient descent
    w1 -= lr * delta_w1
    b1 -= lr * delta_b1
    w2 -= lr * delta_w2
    b2 -= lr * delta_b2
    return w1, b1, w2, b2, loss

def train_epoch(data: Tensor, labels: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, lr: float, batch_size: int) \
        -> Tuple[Tensor, Tensor, Tensor, Tensor, float]:
    non_blocking = w1.device.type == 'cuda'
    epoch_loss = 0
    for i in range(0, data.shape[0], batch_size):
        # Extracting a batch of data and labels
        x = data[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        y = labels[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        w1, b1, w2, b2, batch_loss = train_batch(x, y, w1, b1, w2, b2, lr)
        # Accumulate batch loss to compute epoch loss
        epoch_loss += batch_loss 

        # Compute average epoch loss
        avg_epoch_loss = epoch_loss / batch_size
    return w1, b1, w2, b2, avg_epoch_loss

def evaluate(data: Tensor, labels: Tensor, w1: Tensor, b1: Tensor, w2: Tensor, b2: Tensor, batch_size: int) -> float:
    # Labels are not one hot encoded, because we do not need them as one hot.
    total_correct_predictions = 0
    total_len = data.shape[0]
    non_blocking = w1.device.type == 'cuda'
    for i in range(0, total_len, batch_size):

        # Extract a batch of data and labels
        x = data[i: i + batch_size].to(w1.device, non_blocking=non_blocking)
        y = labels[i: i + batch_size].to(w1.device, non_blocking=non_blocking)

        # Forward pass to get predicted distribution
        predicted_distribution = softmax(forward(sigmoid(forward(x, w1, b1)), w2, b2)) 

        # check torch.max documentation
        predicted_max_value, predicted_max_value_indices = torch.max(predicted_distribution, dim=1)
        # we check if the indices of the max value per line correspond to the correct label. We get a boolean mask
        # with True where the indices are the same, false otherwise
        equality_mask = predicted_max_value_indices == y
        # We sum the boolean mask, and get the number of True values in the mask. We use .item() to get the value out of
        # the tensor
        correct_predictions = equality_mask.sum().item()
        # correct_predictions = (torch.max(predicted_distribution, dim=1)[1] == y).sum().item()
        total_correct_predictions += correct_predictions

    return total_correct_predictions / total_len

def train(epochs: int = 1000, device: torch.device = get_default_device()):
    print(f"Using device {device}")
    pin_memory = device.type == 'cuda'  # Check the provided references.
    # Initialize w and b
    w1 = torch.normal(0, 1 / np.sqrt(784) , (784, 100), device=device) # Xavier/Glorot initialization
    b1 = torch.zeros((1, 100), device=device)
    w2 = torch.normal(0, 1 / np.sqrt(784), (100, 10), device=device)
    b2 = torch.zeros((1, 10), device=device)
    lr = 0.001
    batch_size = 100
    eval_batch_size = 500
    data, labels = load_mnist(train=True, pin_memory=pin_memory)
    data_test, labels_test = load_mnist(train=False, pin_memory=pin_memory)
    # Set up progress bar for epochs
    epochs = tqdm(range(epochs))
    total_loss = 0
    for epoch in epochs:
        epoch_loss = 0
        w1, b1, w2, b2, epoch_loss = train_epoch(data, labels, w1, b1, w2, b2, lr, batch_size)
        total_loss += epoch_loss
        accuracy = evaluate(data_test, labels_test, w1, b1, w2, b2, eval_batch_size)
        # Update progress bar description
        epochs.set_postfix_str(f"accuracy = {accuracy}, epoch_loss = {epoch_loss}, total_loss = {total_loss}")
        if epoch % 100 == 0:
            lr *= 0.90

train(300) # for GPU
train(300, torch.device('cpu'))